## 공시 정보 적재

In [5]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
from sqlalchemy import create_engine, text
import time
import sys
sys.path.append(str(Path.cwd().parent))

from dotenv import load_dotenv
load_dotenv()

DB_USER = os.getenv("POSTGRE_USER")
DB_PASSWORD = os.getenv("POSTGRE_PASSWORD")
DB_HOST = os.getenv("POSTGRE_HOST", "localhost")
DB_PORT = os.getenv("POSTGRE_PORT", "5432")
DB_NAME = os.getenv("POSTGRE_DB")

db_url = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(db_url)
   
# 기업/공시/연구 데이터 수집 모듈
from app.crawler.dart_loader import get_company_info

# 기업 코드
with engine.begin() as conn:
    df_corp_map = pd.read_sql("SELECT corp_code, stock_name FROM dart_corp_map", conn)
    
create_table_sql_company = """
CREATE TABLE IF NOT EXISTS dart_company_info (
    corp_code TEXT PRIMARY KEY,
    corp_name TEXT,
    ceo_name TEXT,
    corp_cls TEXT,
    adres TEXT,
    hm_url TEXT,
    ir_url TEXT
);
"""

create_table_sql_report = """
CREATE TABLE IF NOT EXISTS dart_report_info (
    corp_code TEXT NOT NULL,
    corp_name TEXT,
    stock_code TEXT,
    corp_cls TEXT,
    report_type TEXT,       
    report_year INTEGER,
    report_month INTEGER,
    report_nm TEXT,
    rcept_no TEXT NOT NULL,
    flr_nm TEXT,
    rcept_dt DATE,
    PRIMARY KEY (corp_code, rcept_no)
);
"""

create_table_sql_text = """
CREATE TABLE IF NOT EXISTS dart_report_text (
    corp_code TEXT NOT NULL,
    rcept_no TEXT NOT NULL,
    parsed_url TEXT,            -- 연구개발 섹션 URL
    parsed_table JSONB,         -- 연구개발 실적 테이블
    PRIMARY KEY (corp_code, rcept_no)
);
"""

### 1. 기업 코드 리스트

In [2]:
corp_code_list = list(df_corp_map.corp_code)

### 2. 데이터 수집 시작

In [3]:
collected_company_info = []
collected_report_info = []
collected_report_text = []

for idx, code in enumerate(corp_code_list):

    print(f"\n[{idx+1}/{len(corp_code_list)}] {code} 처리 시작")
    company_info, report_list, rd_text_list = get_company_info(code, start_year="2022")
    
    if company_info:
        collected_company_info.append(company_info)
        print(f"[INFO] 기업 정보 수집 완료: {company_info.get('corp_name','N/A')}")
    else:
        print(f"[SKIP] 기업 정보 없음")

    if report_list:
        collected_report_info.extend(report_list)
        print(f"[INFO] 공시 {len(report_list)}건 수집 완료")
    else:
        print(f"[SKIP] 공시 정보 없음")

    if rd_text_list:
        collected_report_text.extend(rd_text_list)
        print(f"[INFO] 연구개발 문서 {len(rd_text_list)}건 수집 완료")
    else:
        print(f"[SKIP] 연구개발 문서 없음")

[INFO] 연구개발문서 URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20250515001302&dcmNo=10644230&eleId=95&offset=2478168&length=91215&dtd=dart4.xsd

[0/6] 00266961 처리 시작
[INFO] 기업 정보 수집 완료: NAVER
[INFO] 공시 14건 수집 완료
[INFO] 연구개발 문서 1건 수집 완료
[INFO] 연구개발문서 URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20250514001239&dcmNo=10641064&eleId=15&offset=145408&length=33512&dtd=dart4.xsd

[1/6] 00258801 처리 시작
[INFO] 기업 정보 수집 완료: 카카오
[INFO] 공시 15건 수집 완료
[INFO] 연구개발 문서 1건 수집 완료
[INFO] 연구개발문서 URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20250530002640&dcmNo=10671032&eleId=15&offset=36414&length=10410&dtd=dart4.xsd

[2/6] 01547845 처리 시작
[INFO] 기업 정보 수집 완료: 당근마켓
[INFO] 공시 2건 수집 완료
[INFO] 연구개발 문서 1건 수집 완료
{"status":"013","message":"조회된 데이타가 없습니다."}

[3/6] 01717824 처리 시작
[INFO] 기업 정보 수집 완료: 당근페이
[SKIP] 공시 정보 없음
[SKIP] 연구개발 문서 없음
[INFO] 연구개발문서 URL: http://dart.fss.or.kr/report/viewer.do?rcpNo=20250515002618&dcmNo=10647821&eleId=15&offset=69100&length=18975&dtd=dart4.xsd

[4/6] 00126186 처리 시작
[INFO]

### 3. 적재

In [6]:
with engine.begin() as conn:
    conn.execute(text(create_table_sql_company))
    conn.execute(text(create_table_sql_report))
    conn.execute(text(create_table_sql_text))
    
df_company_info = pd.DataFrame(collected_company_info)
df_report_info = pd.DataFrame(collected_report_info)
df_report_text = pd.DataFrame(collected_report_text)

#### 기업정보

In [7]:
df_company_info.to_sql(
    'dart_company_info',
    con=engine,
    if_exists='replace',
    index=False
)

6

#### 공시목록

In [9]:
df_report_info.to_sql(
    'dart_report_info',
    con=engine,
    if_exists='replace',
    index=False
)

59

#### 연구내용

In [10]:
df_report_text.to_sql(
    'dart_report_text',
    con=engine,
    if_exists='replace',
    index=False
)

5

### 4. 적재 확인

In [11]:
query = "SELECT * FROM dart_company_info;"
check = pd.read_sql(query, engine)
check

,corp_code,corp_name,ceo_name,corp_cls,adres,hm_url,ir_url
0,00266961,NAVER,최수연,Y,경기도 성남시 분당구 정자일로 95 1784,www.navercorp.com,https://www.navercorp.com/investment/investors
1,00258801,카카오,정신아,Y,제주특별자치도 제주시 첨단로 242,www.kakaocorp.com,https://www.kakaocorp.com/ir/main
2,01547845,당근마켓,"김용현, 황도연",E,"서울 서초구 강남대로 465 10, 11, 12층",www.daangn.com,about.karrotmarket.com
3,01717824,당근페이,김재현,E,"서울특별시 서초구 강남대로 465 (서초동, 교보생명보험(주)서초사옥)",,
4,00126186,삼성에스디에스,이준희,Y,서울특별시 송파구 올림픽로35길 125(신천동),www.samsungsds.com,
5,00139834,LG씨엔에스,현신균,Y,서울특별시 강서구 마곡중앙8로 71,www.lgcns.com,


In [12]:
query = "SELECT * FROM dart_report_info;"
check2 = pd.read_sql(query, engine)
check2

,corp_code,corp_name,stock_code,corp_cls,report_type,report_year,report_month,report_nm,rcept_no,flr_nm,rcept_dt
0,00266961,NAVER,035420,Y,분기보고서,2025,3,분기보고서 (2025.03),20250515001302,NAVER,2025-05-15
1,00266961,NAVER,035420,Y,사업보고서,2024,12,사업보고서 (2024.12),20250318000645,NAVER,2025-03-18
2,00266961,NAVER,035420,Y,분기보고서,2024,9,분기보고서 (2024.09),20241114001252,NAVER,2024-11-14
3,00266961,NAVER,035420,Y,반기보고서,2024,6,반기보고서 (2024.06),20240814002406,NAVER,2024-08-14
4,00266961,NAVER,035420,Y,분기보고서,2024,3,분기보고서 (2024.03),20240516001974,NAVER,2024-05-16
5,00266961,NAVER,035420,Y,사업보고서,2023,12,사업보고서 (2023.12),20240318000844,NAVER,2024-03-18
6,00266961,NAVER,035420,Y,분기보고서,2023,9,분기보고서 (2023.09),20231114001166,NAVER,2023-11-14
7,00266961,NAVER,035420,Y,반기보고서,2023,6,반기보고서 (2023.06),20230811001363,NAVER,2023-08-11
8,00266961,NAVER,035420,Y,분기보고서,2023,3,분기보고서 (2023.03),20230512001100,NAVER,2023-05-12
9,00266961,NAVER,035420,Y,사업보고서,2022,12,사업보고서 (2022.12),20230314001049,NAVER,2023-03-14


In [13]:
query = "SELECT * FROM dart_report_text;"
check3 = pd.read_sql(query, engine)
check3

,corp_code,rcept_no,parsed_url,parsed_table
0,00266961,20250515001302,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,"{""markdown"": ""4. 연구개발실적\n\n\n| | --- | --- |..."
1,00258801,20250514001239,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,"{""markdown"": ""6. (제조서비스업)주요계약 및 연구개발활동\n\n가. 비..."
2,01547845,20250530002640,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,"{""markdown"": ""6. 주요계약 및 연구개발활동\n\n가. 경영상의 주요계약..."
3,00126186,20250515002618,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,"{""markdown"": ""6. 주요계약 및 연구개발활동\n\n당사의 연구소는 창의와..."
4,00139834,20250515000192,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...,"{""markdown"": ""6. 주요계약 및 연구개발활동\n\n6-1. 경영상의 주요..."


In [17]:
check3.loc[:,'parsed_table'][0]

'{"markdown": "4. 연구개발실적\\n\\n\\n|  |  --- | --- |\\n| 모바일 환경을 위한 초경량화 이미지 인식 기술 | 모바일 환경에서 빠른 속도로 정확하게 인식 가능한 이미지 인식 모델 개발 및 OCR, 얼굴, 사물, 포즈 인식 등의 엔진 및 서비스 적용: ReXNet, ReXNet-Lite 개발완료 및 서비스 적용 중 | 2021 |\\n| 라이브 시청 CDN 트래픽 제어 | 라이브 트래픽 제어가 가능한 재생환경 제공 및 CDN 트래픽 환경개선 | 2021 |\\n| Dolby Atmos Music 스트리밍 인프라 구축 | Dobly Atmos Music 컨텐츠 입수 pipeline 구축, VIBE 의 체험형 오디오 서비스 도입으로 시장 경쟁력에 기여 | 2021 |\\n| 공유 캐시 기반 로딩 성능 최적화 | SRI Cache 스팩 정의, 기술 개발, 웹 표준화 진행을 통하여 웹서비스간 브라우저 캐시 공유를 통한 로딩 성능 개선 | 2021 |\\n| VOD CLOUD 멀티 리전 구축 | 글로벌 환경에서 서비스 품질을 높히고, 리전간 데이터 전송 효율화, 고가용 시스템을 통해 서비스의 글로벌 진출에 따른 경쟁력 확보에 기여ㅈ | 2021 |\\n| 서비스 커스텀 가능한 다크테마 에디터/뷰어 | CSS의 invert 필터와 커스텀 칼라 매핑 테이블을 혼합 개발. 각 서비스의 에디터/뷰어에서 높은 품질의 다크테마를 커스텀할 수 있도록 제공. | 2021 |\\n| LIVE 유료화 DRM 시스템 구축 | VOD 유료화에 이어 LIVE까지 DRM 기반한 유료 서비스 지원을 위한 시스템 구축 | 2021 |\\n| AOD 스토리지 고도화 | Nubes 기반의 글로벌 스토리지 구축, i/o 효율성/성능 개선, 비용 절감 | 2021 |\\n| 라이브 중간광고 제공 | 라이브 진행중 실시간 서버사이드 중간광고 삽입 및 클라이언트 사이드 구글광고 제공 | 2021 |\\n| RAW 이미지 데이터 처리 기술 | 모바일폰에서 RAW 파일 지원 